In [32]:
!pip install pandas numpy matplotlib seaborn pyspark scikit-learn geopandas

In [33]:
""" LIBRAIRIES IMPORTÉES """
import re

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import ArrayType, StringType


In [34]:
""" INITIALISATION """
spark = SparkSession.builder \
    .appName("CorrectionPointsRouges") \
    .master("local[*]") \
    .config("spark.driver.host", "localhost") \
    .getOrCreate()

In [35]:

# 1. Chargement du dataset principal (Points Rouges)
df_pts_rouges = spark.read.option("header", "true") \
    .option("delimiter", ",") \
    .option("multiLine", "true") \
    .option("quote", "\"") \
    .option("escape", "\"") \
    .option("inferSchema", "true") \
    .csv('points_rouges_lyon_complet.csv')

# 2. Chargement du dataset à soustraire (TUNBARJO)
# Attention : le séparateur est un point-virgule ici
df_tunbarjo = spark.read.option("header", "true") \
    .option("delimiter", ";") \
    .option("inferSchema", "true") \
    .csv('TUNBARJO.csv')

# 3. Réalisation du "Left Anti Join" (Soustraction)
# On ne garde que les lignes de pts_rouges qui n'ont PAS de correspondance (long, lat) dans tunbarjo
df_pts_rouges_raw = df_pts_rouges.join(
    df_tunbarjo, 
    on=['longitude', 'latitude'], 
    how='left_anti'
)

# 4. Affichage des résultats et sauvegarde
print(f"Nombre de lignes initiales : {df_pts_rouges.count()}")
print(f"Nombre de lignes finales : {df_pts_rouges_raw.count()}")

df_pts_rouges_raw.show(15)

Nombre de lignes initiales : 24256
Nombre de lignes finales : 22498
+-----------+------------+-------------+--------------------+
|  longitude|    latitude|commune_insee|         description|
+-----------+------------+-------------+--------------------+
|4.836445341|45.758437683|        69123|                NULL|
| 4.85954302|45.731104987|        69123|                NULL|
|4.833717002|45.731325458|        69123|                NULL|
|4.867706711|45.729358056|        69123|                NULL|
|4.797329156|45.803996633|        69194|         Éboulement |
|4.863795432|45.763639697|        69123|Transformation de...|
|4.865337401|45.766321529|        69123|discontinuité des...|
| 4.87557315|45.769984946|        69266|marquages au sol ...|
| 4.87531999|45.771937705|        69266|les voitures pren...|
|4.853396287|45.741487088|        69123|Toute l'Avenue BE...|
|4.874168478|45.738423192|        69123|Rue Maryse Bastié...|
|4.869377621|45.736407856|        69123|Carrefour du Bach...|
| 

In [36]:
""" DIMENSIONS DU DATAFRAME ET VALEURS MANQUANTES PAR COLONNE """
nb_lignes = df_pts_rouges_raw.count()
nb_cols = len(df_pts_rouges_raw.columns)
print(f"Dimensions : ({nb_lignes}, {nb_cols})")

print("--- Valeurs manquantes dans df_pts_rouges après jointure avec les travaux ---")
# On convertit tout en String temporairement pour éviter les erreurs de conversion (Cast)
df_pts_rouges_raw.select([
    F.count(
        F.when(
            F.col(c).isNull() | 
            (F.col(c).cast("string") == "") | 
            (F.col(c).cast("string") == "nan") | 
            (F.col(c).cast("string") == "NaN"), 
            c
        )
    ).alias(c)
    for c in df_pts_rouges_raw.columns
]).show()

Dimensions : (22498, 4)
--- Valeurs manquantes dans df_pts_rouges après jointure avec les travaux ---
+---------+--------+-------------+-----------+
|longitude|latitude|commune_insee|description|
+---------+--------+-------------+-----------+
|        0|       0|            0|       7645|
+---------+--------+-------------+-----------+



In [37]:
""" GESTION DES VALEURS MANQUANTES """

# Supprime toute la ligne si AU MOINS UNE valeur est manquante (NULL ou NaN) dans n'importe quelle colonne
df_pts_rouges_clean = df_pts_rouges_raw.na.drop(how="any")

print(f"Nombre de lignes restantes après nettoyage : {df_pts_rouges_clean.count()}")

Nombre de lignes restantes après nettoyage : 14853


In [38]:
""" DIMENSIONS DU DATAFRAME ET VALEURS MANQUANTES PAR COLONNE """
nb_lignes = df_pts_rouges_clean.count()
nb_cols = len(df_pts_rouges_clean.columns)
print(f"Dimensions : ({nb_lignes}, {nb_cols})")

print("--- Valeurs manquantes dans df_pts_rouges après nettoyage ---")
# On convertit tout en String temporairement pour éviter les erreurs de conversion (Cast)
df_pts_rouges_clean.select([
    F.count(
        F.when(
            F.col(c).isNull() | 
            (F.col(c).cast("string") == "") | 
            (F.col(c).cast("string") == "nan") | 
            (F.col(c).cast("string") == "NaN"), 
            c
        )
    ).alias(c)
    for c in df_pts_rouges_clean.columns
]).show()

# Pour visualiser si le prétraitement a fonctionné 
df_pts_rouges_clean.show(15)

Dimensions : (14853, 4)
--- Valeurs manquantes dans df_pts_rouges après nettoyage ---
+---------+--------+-------------+-----------+
|longitude|latitude|commune_insee|description|
+---------+--------+-------------+-----------+
|        0|       0|            0|          0|
+---------+--------+-------------+-----------+

+-----------+------------+-------------+--------------------+
|  longitude|    latitude|commune_insee|         description|
+-----------+------------+-------------+--------------------+
|4.797329156|45.803996633|        69194|         Éboulement |
|4.863795432|45.763639697|        69123|Transformation de...|
|4.865337401|45.766321529|        69123|discontinuité des...|
| 4.87557315|45.769984946|        69266|marquages au sol ...|
| 4.87531999|45.771937705|        69266|les voitures pren...|
|4.853396287|45.741487088|        69123|Toute l'Avenue BE...|
|4.874168478|45.738423192|        69123|Rue Maryse Bastié...|
|4.869377621|45.736407856|        69123|Carrefour du Bach.

In [39]:
""" CONFIGURATION RÉFÉRENTIELLE """
# Les catégories de problèmes qui serviront de filtre 
CONFIG_PROBLEMES = {
    'Infrastructure manquante': {'mots': r'piste|manque|absence|discontinuité|coupure|cyclable|bande|inexistant', 'urgence': 3, 'facilite': 3},
    'Carrefours dangereux': {'mots': r'carrefour|intersection|rond-point|traversée|giratoire|priorité', 'urgence': 3, 'facilite': 3},
    'Danger & Vitesse': {'mots': r'vitesse|rapide|trop vite|ralentir|dangereu|sécurité|accident|(?<!angle\s)(?<!feuilles\s)mort', 'urgence': 3, 'facilite': 2},
    'Conflit Voiture/Stationnement': {'mots': r'stationnement|garé|voiture|parking|double file|sas|véhicule|portière', 'urgence': 2, 'facilite': 2},
    'Signalisation & Marquage': {'mots': r'panneau|feu|signalisation|marquage|peinture|sol|invisible|effacé', 'urgence': 1, 'facilite': 1},
    'Conflit Piétons': {'mots': r'piéton|trottoir|partagé|quai|mixité|promeneur', 'urgence': 1, 'facilite': 2},
    'Conflit Bus/TCL': {'mots': r'bus|tcl|arrêt|voie bus|angle mort', 'urgence': 2, 'facilite': 2},
    'Aménagement inadapté': {'mots': r'étroit|largeur|bordure|trottoir|poteau|obstacle', 'urgence': 2, 'facilite': 1}
}

def get_label_facilite_txt(score):
    if score <= 1: return "🟢 Facile (Quick Win)"
    if score <= 2: return "🟡 Moyen (Intervention)"
    return "🔴 Difficile (Structurel)"

conf_broadcast = spark.sparkContext.broadcast(CONFIG_PROBLEMES)

# Création du DataFrame de config pour la jointure
data_config = []
for nom, valeurs in CONFIG_PROBLEMES.items():
    data_config.append((nom, valeurs['urgence'], valeurs['facilite'], get_label_facilite_txt(valeurs['facilite'])))
df_config = spark.createDataFrame(data_config, schema=["ref_categorie", "score_urgence", "score_facilite", "niveau_facilite_label"])

In [40]:
""" USER DEFINED FUNCTION (UDF) DE DÉTECTION """
def detecter_themes_py(texte):
    if texte is None or texte == "": return []
    res = []
    for nom, cfg in conf_broadcast.value.items():
        if re.search(cfg['mots'], str(texte), re.IGNORECASE):
            res.append(nom)
    return res

detecter_themes_udf = F.udf(detecter_themes_py, ArrayType(StringType()))

In [41]:
""" TRAITEMENT ET DÉTECTION DES THÈMES """
df_processed = df_pts_rouges_clean.withColumn('problemes_list', detecter_themes_udf(F.col('description')))

In [42]:
""" ENRICHISSEMENT FINAL AVEC SCORES ET CATÉGORIES """
df_exploded = df_processed.select(
    "longitude", 
    "latitude", 
    "commune_insee",
    "description",
    F.explode("problemes_list").alias("categorie_detectee")
)

# Jointure des scores (avec df_config qui doit être défini par ailleurs)
df_gold = df_exploded.join(
    df_config, 
    df_exploded.categorie_detectee == df_config.ref_categorie, 
    "left"
).drop("ref_categorie")

# Visualisation du résultat
df_gold.show(15)

+-----------+------------+-------------+--------------------+--------------------+-------------+--------------+---------------------+
|  longitude|    latitude|commune_insee|         description|  categorie_detectee|score_urgence|score_facilite|niveau_facilite_label|
+-----------+------------+-------------+--------------------+--------------------+-------------+--------------+---------------------+
|4.863795432|45.763639697|        69123|Transformation de...|Conflit Voiture/S...|            2|             2| 🟡 Moyen (Interve...|
| 4.87557315|45.769984946|        69266|marquages au sol ...|Conflit Voiture/S...|            2|             2| 🟡 Moyen (Interve...|
| 4.87531999|45.771937705|        69266|les voitures pren...|Conflit Voiture/S...|            2|             2| 🟡 Moyen (Interve...|
|4.870541611|45.771752747|        69266|Rue Francis de Pr...|Conflit Voiture/S...|            2|             2| 🟡 Moyen (Interve...|
|4.897685708|45.765904369|        69266|route partagée vé...|Confl

In [46]:
""" DIMENSIONS DU DATAFRAME ET VALEURS MANQUANTES PAR COLONNE """
nb_lignes = df_gold.count()
nb_cols = len(df_gold.columns)
print(f"Dimensions : ({nb_lignes}, {nb_cols})")

# Dimensions aggrandit due a explode et donc au duplicata des points
# Pertinent pour garder tous les problèmes détectés par point


Dimensions : (20643, 8)


In [44]:
# Pour visualiser les premières lignes du dataframe gold
df_gold.show(30)

+-----------+------------+-------------+--------------------+--------------------+-------------+--------------+---------------------+
|  longitude|    latitude|commune_insee|         description|  categorie_detectee|score_urgence|score_facilite|niveau_facilite_label|
+-----------+------------+-------------+--------------------+--------------------+-------------+--------------+---------------------+
|4.863795432|45.763639697|        69123|Transformation de...|Conflit Voiture/S...|            2|             2| 🟡 Moyen (Interve...|
| 4.87557315|45.769984946|        69266|marquages au sol ...|Conflit Voiture/S...|            2|             2| 🟡 Moyen (Interve...|
| 4.87531999|45.771937705|        69266|les voitures pren...|Conflit Voiture/S...|            2|             2| 🟡 Moyen (Interve...|
|4.870541611|45.771752747|        69266|Rue Francis de Pr...|Conflit Voiture/S...|            2|             2| 🟡 Moyen (Interve...|
|4.897685708|45.765904369|        69266|route partagée vé...|Confl

In [45]:
""" EXPORT CSV FINAL """
print("Génération du CSV final...")
result_pandas = df_gold.toPandas()
result_pandas = result_pandas.sort_values(by=['score_facilite', 'score_urgence'], ascending=[True, False])

# Export 
output_filename = "points_rouges_sans_travaux_traite.csv"
result_pandas.to_csv(output_filename, index=False, encoding='utf-8-sig')

print("="*80)
print(f"TERMINÉ ! Le fichier '{output_filename}' a été créé.")
print(result_pandas[['categorie_detectee', 'niveau_facilite_label']].head(15).to_string(index=False))

Génération du CSV final...
TERMINÉ ! Le fichier 'points_rouges_sans_travaux_traite.csv' a été créé.
  categorie_detectee niveau_facilite_label
Aménagement inadapté  🟢 Facile (Quick Win)
Aménagement inadapté  🟢 Facile (Quick Win)
Aménagement inadapté  🟢 Facile (Quick Win)
Aménagement inadapté  🟢 Facile (Quick Win)
Aménagement inadapté  🟢 Facile (Quick Win)
Aménagement inadapté  🟢 Facile (Quick Win)
Aménagement inadapté  🟢 Facile (Quick Win)
Aménagement inadapté  🟢 Facile (Quick Win)
Aménagement inadapté  🟢 Facile (Quick Win)
Aménagement inadapté  🟢 Facile (Quick Win)
Aménagement inadapté  🟢 Facile (Quick Win)
Aménagement inadapté  🟢 Facile (Quick Win)
Aménagement inadapté  🟢 Facile (Quick Win)
Aménagement inadapté  🟢 Facile (Quick Win)
Aménagement inadapté  🟢 Facile (Quick Win)
